#### LeetCode - SQL - #262

Refer [262. Trips and Users](https://leetcode.com/problems/trips-and-users/description/)

The  **cancellation rate**  is computed by dividing the number of canceled (by client or driver) requests with unbanned users by the total number of requests with unbanned users on that day.

Write a solution to find the  **cancellation rate**  of requests with unbanned users (**both client and driver must not be banned**) each day between  `"2013-10-01"`  and  `"2013-10-03"`. Round  `Cancellation Rate`  to  **two decimal**  points.

Return the result table in  **any order**.

1. *status in trips is an ENUM (category) type of ('completed', 'cancelled_by_driver', 'cancelled_by_client').*
2. *role in users is an ENUM type of ('client', 'driver', 'partner').*
3. *banned in users is an ENUM (category) type of ('Yes', 'No').*

In [0]:
trips_data = [['1', '1', '10', '1', 'completed', '2013-10-01'], ['2', '2', '11', '1', 'cancelled_by_driver', '2013-10-01'], ['3', '3', '12', '6', 'completed', '2013-10-01'], ['4', '4', '13', '6', 'cancelled_by_client', '2013-10-01'], ['5', '1', '10', '1', 'completed', '2013-10-02'], ['6', '2', '11', '6', 'completed', '2013-10-02'], ['7', '3', '12', '6', 'completed', '2013-10-02'], ['8', '2', '12', '12', 'completed', '2013-10-03'], ['9', '3', '10', '12', 'completed', '2013-10-03'], ['10', '4', '13', '12', 'cancelled_by_driver', '2013-10-03']]
trips_columns = ['id', 'client_id', 'driver_id', 'city_id', 'status', 'request_at']
trips_schema = {'id':'Int64', 'client_id':'Int64', 'driver_id':'Int64', 'city_id':'Int64', 'status':'object', 'request_at':'object'}

users_data = [['1', 'No', 'client'], ['2', 'Yes', 'client'], ['3', 'No', 'client'], ['4', 'No', 'client'], ['10', 'No', 'driver'], ['11', 'No', 'driver'], ['12', 'No', 'driver'], ['13', 'No', 'driver']]
users_columns = ['users_id', 'banned', 'role']
users_schema = {'users_id':'Int64', 'banned':'object', 'role':'object'}

In [0]:
import pandas

trips_pandas_dataframe = pandas.DataFrame(data=trips_data, columns=trips_columns).astype(trips_schema)
users_pandas_dataframe = pandas.DataFrame(data=users_data, columns=users_columns).astype(users_schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
trips_dataframe = spark_context.createDataFrame(trips_pandas_dataframe)
trips_dataframe.printSchema()
trips_dataframe.show(5)

users_dataframe = spark_context.createDataFrame(users_pandas_dataframe)
users_dataframe.printSchema()
users_dataframe.show(5)

root
 |-- id: long (nullable = true)
 |-- client_id: long (nullable = true)
 |-- driver_id: long (nullable = true)
 |-- city_id: long (nullable = true)
 |-- status: string (nullable = true)
 |-- request_at: string (nullable = true)

+---+---------+---------+-------+-------------------+----------+
| id|client_id|driver_id|city_id|             status|request_at|
+---+---------+---------+-------+-------------------+----------+
|  1|        1|       10|      1|          completed|2013-10-01|
|  2|        2|       11|      1|cancelled_by_driver|2013-10-01|
|  3|        3|       12|      6|          completed|2013-10-01|
|  4|        4|       13|      6|cancelled_by_client|2013-10-01|
|  5|        1|       10|      1|          completed|2013-10-02|
+---+---------+---------+-------+-------------------+----------+
only showing top 5 rows

root
 |-- users_id: long (nullable = true)
 |-- banned: string (nullable = true)
 |-- role: string (nullable = true)

+--------+------+------+
|users_id|bann

In [0]:
from pyspark.sql.functions import col, to_date

joined_trips_dataframe \
    = trips_dataframe.alias("trip") \
        .join(users_dataframe.alias("client"), col("trip.client_id") == col("client.users_id"), "inner") \
        .join(users_dataframe.alias("driver"), col("trip.driver_id") == col("driver.users_id"), "inner") \
        .filter((col("client.banned") == "No") & (col("driver.banned") == "No")) \
        .filter((col("trip.request_at") >= "2013-10-01") & (col("trip.request_at") <= "2013-10-03")) \
        .select(col("trip.request_at").alias("request_at"), col("trip.status").alias("status"))
joined_trips_dataframe.show()

+----------+-------------------+
|request_at|             status|
+----------+-------------------+
|2013-10-01|          completed|
|2013-10-01|          completed|
|2013-10-01|cancelled_by_client|
|2013-10-02|          completed|
|2013-10-02|          completed|
|2013-10-03|          completed|
|2013-10-03|cancelled_by_driver|
+----------+-------------------+



In [0]:
from pyspark.sql.functions import count, when, round

result_dataframe \
    = joined_trips_dataframe \
        .groupBy(col("request_at").alias("Day")) \
        .agg( \
            round((count( \
                when(((col("status") == "cancelled_by_client") | (col("status") == "cancelled_by_driver")), 1)) / count("*") \
            ), 2).alias("Cancellation Rate") \
        )
result_dataframe.show()

+----------+-----------------+
|       Day|Cancellation Rate|
+----------+-----------------+
|2013-10-01|             0.33|
|2013-10-02|              0.0|
|2013-10-03|              0.5|
+----------+-----------------+

